In [ ]:
frontal_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
profile_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')

def detect_and_draw_faces_combined(img):
    print("Image shape:", img.shape)
    
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        print("Image is converted into grayscale")
    else:
        gray = img
        print("Is the image already grayscale? It should have only 1 channel:")

    frontal_faces = frontal_face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    profile_faces = profile_face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    flipped_gray = cv2.flip(gray, 1)
    flipped_profile_faces = profile_face_cascade.detectMultiScale(flipped_gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in flipped_profile_faces:
        profile_faces = list(profile_faces)
        profile_faces.append((gray.shape[1] - x - w, y, w, h))
        profile_faces = tuple(profile_faces)

    all_faces = list(frontal_faces) + list(profile_faces)

    unique_faces = []
    for face in all_faces:
        face_tuple = tuple(face)
        if face_tuple not in unique_faces:
            unique_faces.append(face_tuple)

    print(f"Number of faces detected: {len(unique_faces)}")

    if len(unique_faces) > 0:
        for (x, y, w, h) in unique_faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
    else:
        print("No faces detected.")

    if len(img.shape) == 3:
        plt.imshow(img)
        plt.axis('off')
        plt.show()
    else:
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        plt.show()

detect_and_draw_faces_combined(img_cv2)


In [ ]:
cuda_device = dlib.cuda.get_num_devices()
if cuda_device > 0:
    print("CUDA is available. Using GPU for face detection.")
    detector = dlib.cnn_face_detection_model_v1('Models/mmod_human_face_detector.dat')
else:
    print("CUDA is not available. Falling back to CPU.")
    detector = dlib.cnn_face_detection_model_v1('Models/mmod_human_face_detector.dat')

def detect_faces_Dlib(img):
    img_resized = cv2.resize(img, (640, 480))
    gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        x1, y1, x2, y2 = (face.rect.left(), face.rect.top(), face.rect.right(), face.rect.bottom())
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)

    print(f"Number of faces detected: {len(faces)}")

    if len(img.shape) == 3:
        plt.imshow(img)
        plt.axis('off')
        plt.show()
    else:
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        plt.show()
    
    return len(faces)


#detect_faces_Dlib(img_cv2)


In [ ]:
def detect_faces_mtcnn(img, img_index, cropped_faces, default):
    try:
        correct_rotation = 0
        highest_prob = 0.0
        confidence_threshold = 0.80
        length = 4
        horizontal_preferred_dimensions = (1008, 756)
        vertical_preferred_dimensions = (756, 1008)
        valid_boxes = []
        best_probs = []
        found_landmarks = []

        names = [name.strip() for name in df["label_name"][img_index].split(';')]

        for i in range(length):
            empty_box = False
            degrees = 90 * i
            rotated_img = img

            if degrees == 90:
                rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            elif degrees == 180:
                rotated_img = cv2.rotate(img, cv2.ROTATE_180)
            elif degrees == 270:
                rotated_img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

            height, width, _ = rotated_img.shape

            if width > height:
                scale = min(horizontal_preferred_dimensions[0] / width, horizontal_preferred_dimensions[1] / height)
            else:
                scale = min(vertical_preferred_dimensions[0] / width, vertical_preferred_dimensions[1] / height)

            new_width = int(width * scale)
            new_height = int(height * scale)
            resized_img = cv2.resize(rotated_img, (new_width, new_height))

            boxes, probs, landmarks = mtcnn.detect(resized_img, landmarks=True)

            if boxes is None or probs is None or landmarks is None:
                empty_box = True
                boxes, landmarks = [], []

            if not empty_box:
                filtered = [
                    (box, prob, landmark)
                    for box, prob, landmark in zip(boxes, probs, landmarks)
                    if prob >= confidence_threshold
                ]
                boxes, probs, landmarks = zip(*filtered) if filtered else ([], [], [])
                boxes = sorted(boxes, key=lambda box: box[0])

                for prob in probs:
                    if prob > highest_prob:
                        highest_prob = prob
                        best_probs = probs
                        valid_boxes = boxes
                        found_landmarks = landmarks
                        correct_rotation = degrees

        if correct_rotation == 90:
            img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        elif correct_rotation == 180:
            img = cv2.rotate(img, cv2.ROTATE_180)
        elif correct_rotation == 270:
            img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        for box, landmark, prob in zip(valid_boxes, found_landmarks, best_probs):
            height, width, _ = img.shape
            x1, y1, x2, y2 = [int(coord) for coord in box]

            x1 = int(x1 / scale)
            y1 = int(y1 / scale)
            x2 = int(x2 / scale)
            y2 = int(y2 / scale)

            x1 = max(0, min(x1, width - 1))
            y1 = max(0, min(y1, height - 1))
            x2 = max(0, min(x2, width - 1))
            y2 = max(0, min(y2, height - 1))

            cropped_face = img[y1:y2, x1:x2]

            print(cropped_face.shape)

            face_height, face_width, _ = cropped_face.shape
            scale = min(160 / face_width, 160 / face_height)
            resized_width = int(face_width * scale)
            resized_height = int(face_height * scale)
            resized_face = cv2.resize(cropped_face, (resized_width, resized_height))

            padded_face = np.zeros((160, 160, 3), dtype=np.uint8)
            y_offset = (160 - resized_height) // 2
            x_offset = (160 - resized_width) // 2
            padded_face[y_offset:y_offset + resized_height, x_offset:x_offset + resized_width] = resized_face

            cropped_faces.append(padded_face)

            if default:
                default_folder = "Cropped/default"
                os.makedirs(default_folder, exist_ok=True)
            else:
                person_folder = os.path.join("Cropped", names[0])
                os.makedirs(person_folder, exist_ok=True)

            plt.imshow(padded_face)
            plt.axis("off")
            plt.show()

        plt.imshow(img)
        plt.axis("off")
        plt.show()

        return len(valid_boxes)
    except Exception as e:
        print(f"Error detecting faces in image index {img_index}: {e}")
        return 0

In [ ]:
def detect_faces_mtcnn(img, img_index, cropped_faces, default):
    try:
        correct_rotation = 0
        highest_prob = 0.0
        confidence_threshold = 0.80
        length = 4
        horizontal_preferred_dimensions = (1008, 756)
        vertical_preferred_dimensions = (756, 1008)
        valid_boxes = []
        best_probs = []
        found_landmarks = []

        names = [name.strip() for name in df["label_name"][img_index].split(';')]

        for i in range(length):
            empty_box = False
            degrees = 90 * i
            rotated_img = img

            # Rotate the image
            if degrees == 90:
                rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            elif degrees == 180:
                rotated_img = cv2.rotate(img, cv2.ROTATE_180)
            elif degrees == 270:
                rotated_img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

            height, width, _ = rotated_img.shape

            if width > height:
                scale = min(horizontal_preferred_dimensions[0] / width, horizontal_preferred_dimensions[1] / height)
            else:
                scale = min(vertical_preferred_dimensions[0] / width, vertical_preferred_dimensions[1] / height)

            new_width = int(width * scale)
            new_height = int(height * scale)
            resized_img = cv2.resize(rotated_img, (new_width, new_height))

            boxes, probs, landmarks = mtcnn.detect(resized_img, landmarks=True)

            if boxes is None or probs is None or landmarks is None:
                empty_box = True
                boxes, landmarks = [], []

            if not empty_box:
                filtered = [
                    (box, prob, landmark)
                    for box, prob, landmark in zip(boxes, probs, landmarks)
                    if prob >= confidence_threshold
                ]
                boxes, probs, landmarks = zip(*filtered) if filtered else ([], [], [])
                boxes = sorted(boxes, key=lambda box: box[0])

                for prob in probs:
                    if prob > highest_prob:
                        highest_prob = prob
                        best_probs = probs
                        valid_boxes = boxes
                        found_landmarks = landmarks
                        correct_rotation = degrees

        if correct_rotation == 90:
            img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        elif correct_rotation == 180:
            img = cv2.rotate(img, cv2.ROTATE_180)
        elif correct_rotation == 270:
            img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        for box, landmark, prob in zip(valid_boxes, found_landmarks, best_probs):
            height, width, _ = img.shape
            x1, y1, x2, y2 = [int(coord) for coord in box]

            x1 = int(x1 / scale)
            y1 = int(y1 / scale)
            x2 = int(x2 / scale)
            y2 = int(y2 / scale)

            x1 = max(0, min(x1, width - 1))
            y1 = max(0, min(y1, height - 1))
            x2 = max(0, min(x2, width - 1))
            y2 = max(0, min(y2, height - 1))

            cropped_face = img[y1:y2, x1:x2]
            cropped_faces.append(cropped_face)

            if default:
                default_folder = "Cropped/default"
                os.makedirs(default_folder, exist_ok=True)
            else:
                person_folder = os.path.join("Cropped", names[0])
                os.makedirs(person_folder, exist_ok=True)

            #plt.imshow(cropped_face)
            #plt.axis("off")
            #plt.show()

        #plt.imshow(img)
        #plt.axis("off")
        #plt.show()

        return len(valid_boxes)
    except Exception as e:
        print(f"Error detecting faces in image index {img_index}: {e}")
        return 0

In [ ]:
def redetect(img):
    try:
        boxes, _ = mtcnn.detect(img)

        if boxes is None:
            return False

        for box in boxes:
            x1, y1, x2, y2 = [int(coord) for coord in box]
            #cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        return True
    except Exception as e:
        #print(f"Error detecting faces in image: {e}")
        return False

In [ ]:
start = 435
length = 1

for i in range(length):
    cropped_faces = []
    position = i + start
    img = cv2.imread(train_data[position])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
start = 0
length = len(train_data)
correct_detection = 0
Bad_detection = 0
default = False
df_cd = []
df_bd = []

validation = pd.DataFrame(columns=['img', 'person'])

folder_Path = "FaceNet_A"
Check_FilePath = "check"

shape_predictor_path = "shape_predictor_68_face_landmarks.dat"
sp = dlib.shape_predictor(shape_predictor_path)

for i in range(length):
    cropped_faces = []
    position = i + start
    img = cv2.imread(train_data[position])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    detected = detect_faces_mtcnn(img, position, cropped_faces, default)

    img_name = os.path.basename(train_data[position])
    img_name = os.path.splitext(img_name)[0]
    Label_names = [name.strip() for name in df["label_name"][position].split(';')]
    people = len(df["label_name"][position].split(';'))

    if detected == people:
        correct_detection += 1
        df_cd.append(train_data[position])

        for d in range(people):
            try:
                face_img = cropped_faces[d]
                face_img_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)

                face_rect = dlib.rectangle(0, 0, face_img.shape[1], face_img.shape[0])
                shape = sp(face_img_rgb, face_rect)

                aligned_face = dlib.get_face_chip(face_img_rgb, shape, size=150)

                if default:
                    Img_FilePath = os.path.join(folder_Path, "default", f"{img_name}_{d}.jpg")
                else:
                    Img_FilePath = os.path.join(folder_Path, Label_names[d], f"{img_name}_{d}.jpg")

                
                if not os.path.exists(Img_FilePath):
                    os.makedirs(os.path.dirname(Img_FilePath), exist_ok=True)

                cv2.imwrite(Img_FilePath, cv2.cvtColor(aligned_face, cv2.COLOR_RGB2BGR))

                new_row = pd.DataFrame({'img': [Img_FilePath], 'person': [Label_names[d]]})
                validation = pd.concat([validation, new_row], ignore_index=True)

            except Exception as e:
                print(f"Error aligning face {d} in {img_name}: {e}")
    else:
        Bad_detection += 1
        df_bd.append(train_data[position])

    print(f"Processed {i+1}/{length} images")

print(f"{correct_detection}/{len(train_data)} were correctly detected (All faces were detected)")
print(f"{Bad_detection}/{len(train_data)} were incorrectly detected (Not all faces were detected)")